In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import itertools

from scipy import stats
from statsmodels.stats.descriptivestats import sign_test
from statsmodels.stats.weightstats import zconfint
from statsmodels.stats import weightstats

#### 4 вопрос

In [3]:
limfoma = np.array([49,58,75,110,112,132,151,276,281,362])
stats.wilcoxon(limfoma - 200)

WilcoxonResult(statistic=17.0, pvalue=0.28450269791120752)

#### 5 вопрос

In [12]:
good = np.array([22,22,15,13,19,19,18,20,21,13,13,15])
bad  = np.array([17,18,18,15,12,4,14,15,10])
stats.mannwhitneyu(good, bad, alternative='greater')

MannwhitneyuResult(statistic=81.0, pvalue=0.029004992720873729)

In [17]:
data = pd.read_csv("challenger.txt", delimiter="\t", header=0,index_col=0,)

In [18]:
data.head()

,Temperature,Incident
Apr12.81,18.9,0
Nov12.81,21.1,1
Mar22.82,20.6,0
Nov11.82,20.0,0
Apr04.83,19.4,0


In [19]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [20]:
def stat_intervals(stat, alpha = 0.05):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [21]:
incident = data.Temperature[data.Incident == 1]
clear = data.Temperature[data.Incident == 0]

np.random.seed(0)
clear_means = map(np.mean, get_bootstrap_samples(clear, 1000))
incident_means = map(np.mean, get_bootstrap_samples(incident, 1000))

delta_means = map(lambda x: x[0] - x[1], zip(clear_means, incident_means))
stat_intervals(delta_means)

array([ 1.42299107,  7.93861607])

In [22]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

In [23]:
def get_random_combinations(n1, n2, max_combinations):
    index = range(n1 + n2)
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

In [24]:
def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

In [25]:
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [26]:
np.random.seed(0)
permutation_test(incident, clear, max_permutations = 10000)

0.0057